In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.models as models
from torchvision import transforms
import os

In [ ]:
train_data_path = "../../data/cat_and_dog/train"
test_data_path = "../../data/cat_and_dog/validation"

model_path = '../../model/cat_and_dog/'

LEARNING_RATE = 5e-4
EPOCHS = 30
BATCH_SIZE = 50

In [ ]:
class CatAndDog(nn.Module):
    def __init__(self, ic=3, channels=(32, 64, 32, 8), kernels=(3, 3, 3, 3)):
        super(CatAndDog, self).__init__()

        self.conv_layers = nn.ModuleList()

        in_ch = ic
        for c, k in zip(channels, kernels):
            self.conv_layers.append(nn.Conv2d(in_ch, c, k))
            self.conv_layers.append(nn.BatchNorm2d(c))
            self.conv_layers.append(nn.ReLU())
            self.conv_layers.append(nn.MaxPool2d(2))
            in_ch = c

        self.linear_layers = nn.ModuleList()
        self.linear_layers.append(nn.Linear(8*14*14, 100))
        self.linear_layers.append(nn.ReLU())
        self.linear_layers.append(nn.Linear(100, 2))
        # self.linear_layers.append(nn.Softmax())

    def forward(self, x):
        for layer in self.conv_layers:
            x = layer(x)
        x = x.reshape(x.size(0), -1)

        for layer in self.linear_layers:
            x = layer(x)
        return x

In [ ]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(256),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def get_loader(path, transform, batch_size):
    dataset = torchvision.datasets.ImageFolder(
        root = path,
        transform=transform
    )
    loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=batch_size,
        num_workers=0,
        shuffle=True
    )
    return loader

In [ ]:
train_loader = get_loader(train_data_path, transform, BATCH_SIZE)

In [ ]:
criteria = F.cross_entropy
module = CatAndDog().cuda()

optimizer = torch.optim.Adam(module.parameters(), lr=LEARNING_RATE)

loss_sum = torch.tensor(0, dtype=torch.float32)
for epoch in range(EPOCHS):
    print("{} epoch".format(epoch))
    for x, y in train_loader:
        output = module(x.cuda())
        loss = criteria(output, y.cuda())

        loss_sum += loss.detach().item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print("\t{}".format(loss_sum.mean()))
    loss_sum = torch.tensor(0, dtype=torch.float32)

In [ ]:
torch.save(module.state_dict(), os.path.join(model_path, 'custom.pth'))

In [ ]:
# module = CatAndDog()
# module.load_state_dict(torch.load(os.path.join(model_path, 'custom.pth')))
# module = module.cuda()

In [ ]:
test_loader = get_loader(test_data_path, transform, BATCH_SIZE)

total = correct = 0
for x, y in test_loader:
    output = module(x.cuda())

    val, pred_idx = output.max(1)
    correct += (pred_idx == y.cuda()).sum().item()
    total += val.shape[0]

print ("correct : {} // total : {}".format(correct, total))
print ("accuracy: {}".format(correct / total))

<pre>
0 epoch
	26.269121170043945
1 epoch
	22.54401397705078
2 epoch
	19.70692253112793
3 epoch
	17.80558967590332
4 epoch
	15.526264190673828
5 epoch
	14.089925765991211
6 epoch
	13.086111068725586
7 epoch
	11.234223365783691
8 epoch
	9.514725685119629
9 epoch
	8.090290069580078
10 epoch
	7.891892433166504
11 epoch
	6.033896446228027
12 epoch
	5.001809120178223
13 epoch
	4.299685001373291
14 epoch
	4.592019557952881
15 epoch
	3.125213861465454
16 epoch
	1.906551718711853
17 epoch
	2.056344985961914
18 epoch
	2.333315372467041
19 epoch
	1.6180298328399658
20 epoch
	1.0541620254516602
21 epoch
	0.9271639585494995
22 epoch
	0.8449956178665161
23 epoch
	0.5367169976234436
24 epoch
	0.347711980342865
25 epoch
	0.23818166553974152
26 epoch
	0.22533170878887177
27 epoch
	0.2599599063396454
28 epoch
	0.1610652059316635
29 epoch
	0.1274682730436325
    
correct : 602 // total : 804
accuracy: 0.7487562189054726
</pre>